In [3]:
# Single dataset
import arviz as az  # Visualization
import bambi as bmb  #
import hssm
import csv
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import os
import matplotlib.pyplot as plt
import pymc as pm
import requests
from io import StringIO
from joblib import Parallel, delayed
from hssm.distribution_utils import make_distribution
from hssm.likelihoods import DDM

In [4]:
# Nextcloud credentials
username = 'algarrid'
password = 'faunistico'

exp_version = 2

## Clean cwir_task_data experiment data

In [6]:
# URL of the CSV file you want to access (use the full path to the file)

if exp_version == 2:
    file_url = (
        'https://cloud.ilabt.imec.be/remote.php/dav/files/'+
        'af741990-37f9-103d-9441-9bec5c4808a7/ExperimentsData/'+
        'risky_dm_'+str(exp_version)+'/ExperimentData/experiment_reprotaskmethod1quest1_answers.csv'            
    )
else: 
    file_url = (
        'https://cloud.ilabt.imec.be/remote.php/dav/files/'+
        'af741990-37f9-103d-9441-9bec5c4808a7/ExperimentsData/'+
        'risky_dm_'+str(exp_version)+'/pilot/experiment_rounds_answers.csv'            
    )
# Make a request to get the file content
response = requests.get(file_url, auth=(username, password))

# Check if the request was successful
if response.status_code == 200:
    # Stream the content directly into a pandas DataFrame
    csv_content = StringIO(response.text)
    df_experiment = pd.read_csv(csv_content)

    # Now `df` is your DataFrame containing the CSV data
    print("Data downloaded")  # Example: print the first few rows
else:
    print(f"Failed to retrieve the file. Status code: {response.status_code}")
    
#Fix bug of pilot number 2
if exp_version == 2:
    df_experiment.loc[
        (df_experiment['ResponseTimeGamble'] > 8) & (df_experiment['GambleFinalDeadline'] == 8), 
        'GambleFinalDeadline'
    ] = 10

if exp_version == 2:
    df_experiment.loc[
        df_experiment['MissedMaxDeadline'] ==1, 
        'GambleFinalDeadline'
    ] = df_experiment.loc[
        df_experiment['MissedMaxDeadline'] ==1, 
        'ResponseTimeGamble']
    

    


Data downloaded


/tmp/ipykernel_9672/142690703.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[10.6592 11.3383 19.671  11.823  10.5689 12.6928 11.3842 15.147  14.7072
 10.3309 16.2714 10.045  12.1129 10.0788 11.1289 12.475  22.8211 10.243
 16.8971 10.0673 11.338  12.5007 14.9441 11.0494 13.6039 14.1023 15.6531
 19.1477 12.8313 10.326  11.32   11.223  10.664  11.7561 10.1589 23.3356]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_experiment.loc[


In [7]:
rejected_participants_list = ['670ca14484c2bdc633b10f21', # Extremely low Reproduced Times and failed both attention checks
                                  '6691b722e54f9a743dfd308c', # This participant failed the comprehension check according to prolific so not even sure how he managed to pass the thing
                                  '6658ca018e28e3383feae8e9', # This one exited the full screen mode more than 3 times, and only did 24 minutes
                                  '62ac71cec3309a48781684eb', #exited full screen mode more than three times
                             ]

df_experiment = df_experiment[~df_experiment['ParticipantID'].isin(rejected_participants_list)] # Remove all rejected participants

par_ids = np.unique(df_experiment['ParticipantID'])
n_par = len(par_ids)

anom_ids = np.arange(1,len(par_ids)+1)

# par_ids and anom_ids are lists/arrays of strings
par_ids = np.asarray(par_ids, dtype=object)
anom_ids = np.asarray(anom_ids, dtype=object)

par_ids_trans = np.empty((2, len(par_ids)), dtype=object)
par_ids_trans[0, :] = par_ids
par_ids_trans[1, :] = anom_ids

id_map = dict(zip(par_ids, anom_ids))

In [97]:
print(id_map['622e282986d38f7f8fb74e93'])
print(id_map['6658b14e39f5e53a96a6a8f4'])

14
19


In [9]:
# Replace participant IDs
df_experiment["ParticipantID"] = df_experiment["ParticipantID"].replace(id_map)


/tmp/ipykernel_16993/2274876970.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_experiment["ParticipantID"] = df_experiment["ParticipantID"].replace(id_map)


In [ ]:
df_experiment

In [8]:


# 1) Drop the first three columns
cols_to_drop = ["SessionID", "StudyID", "ExperimentRoundID"]
df2 = df_experiment.drop(columns=cols_to_drop, errors="ignore")

# 2) Ensure numeric sort keys, then sort by participant → round → (optionally) gamble
df2["ExperimentRoundNumber"] = pd.to_numeric(df2["ExperimentRoundNumber"], errors="coerce")
# If ParticipantID is numeric-like (e.g., 1..23); if it's strings like "P001", remove this next line
df2["ParticipantID"] = pd.to_numeric(df2["ParticipantID"], errors="ignore")

df2 = df2.sort_values(
    by=["ParticipantID", "ExperimentRoundNumber"],
    kind="mergesort"  # stable sort keeps within-round row order if ties
).reset_index(drop=True)

# 3) Reorder columns: put key columns first, keep the rest as-is
first = ["ParticipantID", "BlockNumber", "ExperimentRoundNumber", "ExperimentGambleNumber"]
df2 = df2[first + [c for c in df2.columns if c not in first]]

# (optional) sanity checks
# print(df2.head(12))
# df2.to_csv("anonymized_sorted.csv", index=False)



/tmp/ipykernel_9672/3211685806.py:8: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df2["ParticipantID"] = pd.to_numeric(df2["ParticipantID"], errors="ignore")


In [11]:
df2

,ParticipantID,BlockNumber,ExperimentRoundNumber,ExperimentGambleNumber,LotteryLeft,LotteryRight,GambleChoice,ResponseTimeGamble,GambleSecondaryChoices,GambleSecondaryChoicesTimes,GamblePresetDeadline,GambleFinalDeadline,FullScreenExited,ReproducedTime,MissedMaxDeadline,SubjectiveDifficulty,ResponseTimeSubjectiveDifficulty,ResponseTimeSubmitAnswersQuestionnaire
0,1,1,1,135,1,0,Left,5.0060,NaN,NaN,6,6,0,3.7731,0,3,1.8775,5.4138
1,1,1,2,19,1,0,Left,5.4893,NaN,NaN,8,8,0,3.4387,0,3,4.4075,5.7875
2,1,1,3,112,1,0,Right,3.4687,NaN,NaN,6,6,0,5.0266,0,2,1.1633,3.4060
3,1,1,4,89,1,0,Right,5.2408,NaN,NaN,8,8,0,5.4989,0,3,1.6638,4.0125
4,1,1,5,99,0,1,Left,6.3014,NaN,NaN,6,8,0,5.8856,0,3,4.2498,5.8229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3667,23,5,156,70,0,1,Right,2.2655,NaN,NaN,8,8,0,38.0002,0,2,2.6439,3.5947
3668,23,5,157,141,1,0,Right,2.2443,NaN,NaN,6,6,0,10.2145,0,3,3.2333,4.0368
3669,23,5,158,102,0,1,Right,2.1185,NaN,NaN,10,10,0,5.5257,0,2,2.0589,2.9075
3670,23,5,159,26,1,0,Left,2.4010,NaN,NaN,6,6,0,11.6741,0,4,2.2274,3.0082


In [16]:
df_experiment

,StudyID,SessionID,ExperimentRoundID,ParticipantID,BlockNumber,ExperimentRoundNumber,ExperimentGambleNumber,LotteryLeft,LotteryRight,GambleChoice,...,GambleSecondaryChoices,GambleSecondaryChoicesTimes,GamblePresetDeadline,GambleFinalDeadline,FullScreenExited,ReproducedTime,MissedMaxDeadline,SubjectiveDifficulty,ResponseTimeSubjectiveDifficulty,ResponseTimeSubmitAnswersQuestionnaire
0,670cd6fec4b9b923bf4d92dd,670cd9c5c773e51a8840bad7,590df20e12802600017e1006_1_experiment,1,3,66,1,1,0,Right,...,NaN,NaN,6,6,0,5.0358,0,2,1.2011,2.7060
1,670cd6fec4b9b923bf4d92dd,670cd9c5c773e51a8840bad7,590df20e12802600017e1006_10_experiment,1,1,16,10,0,1,Right,...,NaN,NaN,10,10,0,8.3527,0,2,4.0082,5.5249
2,670cd6fec4b9b923bf4d92dd,670cd9c5c773e51a8840bad7,590df20e12802600017e1006_100_experiment,1,5,142,100,0,1,Left,...,NaN,NaN,6,10,0,11.1855,1,5,1.0142,2.4017
3,670cd6fec4b9b923bf4d92dd,670cd9c5c773e51a8840bad7,590df20e12802600017e1006_101_experiment,1,2,53,101,0,1,Left,...,NaN,NaN,8,8,0,6.7921,0,2,1.1883,4.8483
4,670cd6fec4b9b923bf4d92dd,670cd9c5c773e51a8840bad7,590df20e12802600017e1006_102_experiment,1,4,100,102,0,1,Left,...,NaN,NaN,10,10,0,13.5556,0,2,2.5908,7.7971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3923,670cd6fec4b9b923bf4d92dd,670cd91883a96ff37fb79de2,670a165e0ef22b0779d470dd_95_experiment,23,3,70,95,0,1,Left,...,NaN,NaN,10,10,0,20.2493,0,3,2.3500,5.5026
3924,670cd6fec4b9b923bf4d92dd,670cd91883a96ff37fb79de2,670a165e0ef22b0779d470dd_96_experiment,23,1,31,96,0,1,Left,...,NaN,NaN,6,6,0,8.2278,0,2,1.7716,2.7794
3925,670cd6fec4b9b923bf4d92dd,670cd91883a96ff37fb79de2,670a165e0ef22b0779d470dd_97_experiment,23,2,54,97,1,0,Right,...,NaN,NaN,6,6,0,9.1522,0,2,2.3513,3.8363
3926,670cd6fec4b9b923bf4d92dd,670cd91883a96ff37fb79de2,670a165e0ef22b0779d470dd_98_experiment,23,5,131,98,0,1,Left,...,NaN,NaN,6,6,0,6.5219,0,2,2.0366,2.9777


In [12]:
df2.to_csv("cwir_task_data.csv", index=False)

# practice trials behavioural data

In [83]:
if exp_version == 2:
    file_url = (
        'https://cloud.ilabt.imec.be/remote.php/dav/files/'+
        'af741990-37f9-103d-9441-9bec5c4808a7/ExperimentsData/'+
        'risky_dm_'+str(exp_version)+'/ExperimentData/practice_reprotaskmethod1quest1_answers.csv'            
    )
else: 
    file_url = (
        'https://cloud.ilabt.imec.be/remote.php/dav/files/'+
        'af741990-37f9-103d-9441-9bec5c4808a7/ExperimentsData/'+
        'risky_dm_'+str(exp_version)+'/pilot/practice_reprotaskmethod1quest1_answers.csv'            
    )
# Make a request to get the file content
response = requests.get(file_url, auth=(username, password))

# Check if the request was successful
if response.status_code == 200:
    # Stream the content directly into a pandas DataFrame
    csv_content = StringIO(response.text)
    df_practice = pd.read_csv(csv_content)

    # Now `df` is your DataFrame containing the CSV data
    print("Practice Data downloaded")  # Example: print the first few rows
else:
    print(f"Failed to retrieve the file. Status code: {response.status_code}")

Practice Data downloaded


In [85]:
df_practice = df_practice[df_practice['ParticipantID'].isin(par_ids)]

df_practice["ParticipantID"] = df_practice["ParticipantID"].replace(id_map)

/tmp/ipykernel_16993/2062698711.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_practice["ParticipantID"] = df_practice["ParticipantID"].replace(id_map)


In [86]:
df_practice

,StudyID,SessionID,PracticeRoundID,ParticipantID,PracticeGambleNumber,GambleChoice,ResponseTimeGamble,GambleSecondaryChoices,GambleSecondaryChoicesTimes,GamblePresetDeadline,MissedDeadline,FullScreenExited,ReproducedTime,SubjectiveDifficulty,ResponseTimeSubjectiveDifficulty,ResponseTimeSubmitAnswersQuestionnaire
0,670cd6fec4b9b923bf4d92dd,670cd9c5c773e51a8840bad7,590df20e12802600017e1006_1_practice,1,1,Left,4.7753,NaN,NaN,6,0,0,4450.3,3,7.7463,10.6484
1,670cd6fec4b9b923bf4d92dd,670cd9c5c773e51a8840bad7,590df20e12802600017e1006_2_practice,1,2,Left,3.0241,NaN,NaN,8,0,0,7986.0,2,4.6049,6.5986
2,670cd6fec4b9b923bf4d92dd,670cd9c5c773e51a8840bad7,590df20e12802600017e1006_3_practice,1,3,Left,3.9237,NaN,NaN,10,0,0,7620.1,3,6.7020,9.0688
3,670cd6fec4b9b923bf4d92dd,670cd9c5c773e51a8840bad7,590df20e12802600017e1006_4_practice,1,4,Left,5.4500,NaN,NaN,8,0,0,4838.4,3,1.8443,4.1436
4,670cd6fec4b9b923bf4d92dd,670cd9c5c773e51a8840bad7,590df20e12802600017e1006_5_practice,1,5,Left,3.1528,NaN,NaN,6,0,0,4683.6,3,2.5958,4.7149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,670cd6fec4b9b923bf4d92dd,670cd91883a96ff37fb79de2,670a165e0ef22b0779d470dd_2_practice,23,2,Right,2.4482,NaN,NaN,8,0,0,4656.3,3,2.4595,3.6494
160,670cd6fec4b9b923bf4d92dd,670cd91883a96ff37fb79de2,670a165e0ef22b0779d470dd_3_practice,23,3,Left,2.1757,NaN,NaN,10,0,0,6122.5,4,2.1842,3.1024
161,670cd6fec4b9b923bf4d92dd,670cd91883a96ff37fb79de2,670a165e0ef22b0779d470dd_4_practice,23,4,Left,2.3343,NaN,NaN,8,0,0,7808.4,4,2.7401,3.7151
162,670cd6fec4b9b923bf4d92dd,670cd91883a96ff37fb79de2,670a165e0ef22b0779d470dd_5_practice,23,5,Left,3.3430,NaN,NaN,6,0,0,5802.7,2,2.0995,3.5044


In [87]:
cols_to_drop = ["SessionID", "StudyID", "PracticeRoundID"]
df2 = df_practice.drop(columns=cols_to_drop, errors="ignore")



In [88]:
df2

,ParticipantID,PracticeGambleNumber,GambleChoice,ResponseTimeGamble,GambleSecondaryChoices,GambleSecondaryChoicesTimes,GamblePresetDeadline,MissedDeadline,FullScreenExited,ReproducedTime,SubjectiveDifficulty,ResponseTimeSubjectiveDifficulty,ResponseTimeSubmitAnswersQuestionnaire
0,1,1,Left,4.7753,NaN,NaN,6,0,0,4450.3,3,7.7463,10.6484
1,1,2,Left,3.0241,NaN,NaN,8,0,0,7986.0,2,4.6049,6.5986
2,1,3,Left,3.9237,NaN,NaN,10,0,0,7620.1,3,6.7020,9.0688
3,1,4,Left,5.4500,NaN,NaN,8,0,0,4838.4,3,1.8443,4.1436
4,1,5,Left,3.1528,NaN,NaN,6,0,0,4683.6,3,2.5958,4.7149
...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,23,2,Right,2.4482,NaN,NaN,8,0,0,4656.3,3,2.4595,3.6494
160,23,3,Left,2.1757,NaN,NaN,10,0,0,6122.5,4,2.1842,3.1024
161,23,4,Left,2.3343,NaN,NaN,8,0,0,7808.4,4,2.7401,3.7151
162,23,5,Left,3.3430,NaN,NaN,6,0,0,5802.7,2,2.0995,3.5044


In [89]:
df2.to_csv("practice_trials.csv", index=False)

## Block end questionnaire answers

In [42]:
# Define the path to your CSV file
if exp_version  == 2:
    file_url = (
        'https://cloud.ilabt.imec.be/remote.php/dav/files/af741990-37f9-103d-9441-9bec5c4808a7/'+
        'ExperimentsData/risky_dm_'+str(exp_version)+'/ExperimentData/block_end_questionnaire_answers.csv'
    )
    
else:
    file_url = (
        'https://cloud.ilabt.imec.be/remote.php/dav/files/af741990-37f9-103d-9441-9bec5c4808a7/'+
        'ExperimentsData/risky_dm_'+str(exp_version)+'/pilot/block_end_questionnaire_answers.csv'
    )

# Make a request to get the file content
response = requests.get(file_url, auth=(username, password))

# Check if the request was successful
if response.status_code == 200:
    # Stream the content directly into a pandas DataFrame
    csv_content = StringIO(response.text)
    df_block_end = pd.read_csv(csv_content)

    # Now `df` is your DataFrame containing the CSV data
    print("Block end questionnaire downloaded")  # Example: print the first few rows
else:
    print(f"Failed to retrieve the file. Status code: {response.status_code}")
    
df_block_end = df_block_end[df_block_end['ParticipantID'].isin(par_ids)]

# Display the first few rows of the DataFrame
print(df_block_end.columns)

Block end questionnaire downloaded
Index(['StudyID', 'SessionID', 'ParticipantID', 'ParticipantID_BlockNumber',
       'BlockNumber', 'SubjectiveBlockTotalTime',
       'ResponseTimeSubjectiveBlockTotalTime',
       'ResponseTimeFirstAttemptSubjectiveBlockTotalTime',
       'AttemptsAnswerSubjectiveBlockTotalTime',
       'ConfidenceSubjectiveBlockTotalTime',
       'ResponseTimeConfidenceSubjectiveBlockTotalTime', 'LevelBoredom',
       'ResponseTimeLevelBoredom', 'TotalObjectiveBlockTime'],
      dtype='object')


In [43]:
# Replace participant IDs
 

df_block_end["ParticipantID"] = df_block_end["ParticipantID"].replace(id_map)

# 1) Drop the first three columns
cols_to_drop = ["SessionID", "StudyID", 'ParticipantID_BlockNumber']
df2_block_end = df.drop(columns=cols_to_drop, errors="ignore")

# 2) Ensure numeric sort keys, then sort by participant → round → (optionally) gamble
df2_block_end['BlockNumber'] = pd.to_numeric(df2_block_end['BlockNumber'], errors="coerce")
# If ParticipantID is numeric-like (e.g., 1..23); if it's strings like "P001", remove this next line
df2_block_end["ParticipantID"] = pd.to_numeric(df2_block_end["ParticipantID"], errors="ignore")

df2_block_end = df2_block_end.sort_values(
    by=["ParticipantID", 'BlockNumber'],
    kind="mergesort"  # stable sort keeps within-round row order if ties
).reset_index(drop=True)

# 3) Reorder columns: put key columns first, keep the rest as-is
first = ["ParticipantID", "BlockNumber"]
df2_block_end = df2_block_end[first + [c for c in df2_block_end.columns if c not in first]]

/tmp/ipykernel_16993/4212723948.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_block_end["ParticipantID"] = df_block_end["ParticipantID"].replace(id_map)
/tmp/ipykernel_16993/4212723948.py:13: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df2_block_end["ParticipantID"] = pd.to_numeric(df2_block_end["ParticipantID"], errors="ignore")


In [44]:
df2_block_end.columns

Index(['ParticipantID', 'BlockNumber', 'SubjectiveBlockTotalTime',
       'ResponseTimeSubjectiveBlockTotalTime',
       'ResponseTimeFirstAttemptSubjectiveBlockTotalTime',
       'AttemptsAnswerSubjectiveBlockTotalTime',
       'ConfidenceSubjectiveBlockTotalTime',
       'ResponseTimeConfidenceSubjectiveBlockTotalTime', 'LevelBoredom',
       'ResponseTimeLevelBoredom', 'TotalObjectiveBlockTime'],
      dtype='object')

In [46]:
df2_block_end

,ParticipantID,BlockNumber,SubjectiveBlockTotalTime,ResponseTimeSubjectiveBlockTotalTime,ResponseTimeFirstAttemptSubjectiveBlockTotalTime,AttemptsAnswerSubjectiveBlockTotalTime,ConfidenceSubjectiveBlockTotalTime,ResponseTimeConfidenceSubjectiveBlockTotalTime,LevelBoredom,ResponseTimeLevelBoredom,TotalObjectiveBlockTime
0,1,1,15,17.1450,NaN,NaN,3,22.6156,3,30.8356,796.3020
1,1,2,18,8.7234,0.0,1.0,4,13.1538,4,15.4898,854.8375
2,1,3,14,8.3456,0.0,1.0,3,9.2773,3,12.2539,691.7451
3,1,4,16,9.6975,0.0,1.0,3,10.5571,3,12.1738,870.5918
4,1,5,17,8.4852,0.0,2.0,3,10.5948,3,11.8684,743.7520
...,...,...,...,...,...,...,...,...,...,...,...
110,23,1,15,38.3377,NaN,NaN,3,18.0584,3,21.6701,852.6189
111,23,2,20,8.6119,0.0,2.0,2,18.1726,2,23.4319,986.9972
112,23,3,20,9.3456,0.0,2.0,2,12.3018,2,16.6771,935.9554
113,23,4,25,8.9083,0.0,1.0,4,24.0184,4,25.9098,1151.2454


In [45]:
df2_block_end.to_csv("block_end_questionnaire_answers.csv", index=False)

## Breaks_data

In [47]:
# Define the path to your CSV file
if exp_version == 2:
    file_url = (
        'https://cloud.ilabt.imec.be/remote.php/dav/files/'+
        'af741990-37f9-103d-9441-9bec5c4808a7/ExperimentsData/'+
        'risky_dm_'+str(exp_version)+'/ExperimentData/breaks_data.csv'
    )
    
else:
    file_url = (
        'https://cloud.ilabt.imec.be/remote.php/dav/files/'+
        'af741990-37f9-103d-9441-9bec5c4808a7/ExperimentsData/'+
        'risky_dm_'+str(exp_version)+'/pilot/breaks_data.csv'
    )

# Make a request to get the file content
response = requests.get(file_url, auth=(username, password))

# Check if the request was successful
if response.status_code == 200:
    # Stream the content directly into a pandas DataFrame
    csv_content = StringIO(response.text)
    df_breaks = pd.read_csv(csv_content)

    # Now `df` is your DataFrame containing the CSV data
    print("Breaks data downloaded")  # Example: print the first few rows
else:
    print(f"Failed to retrieve the file. Status code: {response.status_code}")
df_breaks = df_breaks[df_breaks['ParticipantID'].isin(par_ids)]
# Display the first few rows of the DataFrame
print(df_breaks.columns)

Breaks data downloaded
Index(['StudyID', 'SessionID', 'ParticipantID', 'ParticipantID_BlockNumber',
       'BlockNumber', 'BreakTimeDuration', 'BreakTime'],
      dtype='object')


In [48]:
# Replace participant IDs

df = df_breaks
df["ParticipantID"] = df["ParticipantID"].replace(id_map)

# 1) Drop the first three columns
cols_to_drop = ["SessionID", "StudyID", 'ParticipantID_BlockNumber']
df2 = df.drop(columns=cols_to_drop, errors="ignore")

# 2) Ensure numeric sort keys, then sort by participant → round → (optionally) gamble
df2['BlockNumber'] = pd.to_numeric(df2['BlockNumber'], errors="coerce")
# If ParticipantID is numeric-like (e.g., 1..23); if it's strings like "P001", remove this next line
df2["ParticipantID"] = pd.to_numeric(df2["ParticipantID"], errors="ignore")

df2 = df2.sort_values(
    by=["ParticipantID", 'BlockNumber'],
    kind="mergesort"  # stable sort keeps within-round row order if ties
).reset_index(drop=True)

# 3) Reorder columns: put key columns first, keep the rest as-is
first = ["ParticipantID", "BlockNumber"]
df2 = df2[first + [c for c in df2.columns if c not in first]]

/tmp/ipykernel_16993/3018918608.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["ParticipantID"] = df["ParticipantID"].replace(id_map)
/tmp/ipykernel_16993/3018918608.py:13: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df2["ParticipantID"] = pd.to_numeric(df2["ParticipantID"], errors="ignore")


In [49]:
df_breaks

,StudyID,SessionID,ParticipantID,ParticipantID_BlockNumber,BlockNumber,BreakTimeDuration,BreakTime
0,670cd6fec4b9b923bf4d92dd,670cd9c5c773e51a8840bad7,1,590df20e12802600017e1006_2,2,213.0306,2606.8969
1,670cd6fec4b9b923bf4d92dd,670cd9c5c773e51a8840bad7,1,590df20e12802600017e1006_3,3,236.7673,3698.5074
2,670cd6fec4b9b923bf4d92dd,670cd9c5c773e51a8840bad7,1,590df20e12802600017e1006_4,4,228.3141,4618.5674
3,670cd6fec4b9b923bf4d92dd,670cd9c5c773e51a8840bad7,1,590df20e12802600017e1006_5,5,182.5863,5671.7441
4,66d9bf36bfbd262851fca662,67039f799780ebcf741bc3df,2,5ae33e48ed7f8b0001ca7bf8_2,2,217.0071,1587.1896
...,...,...,...,...,...,...,...
92,66d9bf36bfbd262851fca662,67039ceef6f58d59045de43b,22,66feee5066f9e3e110cac5d8_5,5,238.0134,5274.0213
93,670cd6fec4b9b923bf4d92dd,670cd91883a96ff37fb79de2,23,670a165e0ef22b0779d470dd_2,2,183.8715,1661.2590
94,670cd6fec4b9b923bf4d92dd,670cd91883a96ff37fb79de2,23,670a165e0ef22b0779d470dd_3,3,182.1562,2830.4074
95,670cd6fec4b9b923bf4d92dd,670cd91883a96ff37fb79de2,23,670a165e0ef22b0779d470dd_4,4,189.5532,3955.9252


In [50]:
df2

,ParticipantID,BlockNumber,BreakTimeDuration,BreakTime
0,1,2,213.0306,2606.8969
1,1,3,236.7673,3698.5074
2,1,4,228.3141,4618.5674
3,1,5,182.5863,5671.7441
4,2,2,217.0071,1587.1896
...,...,...,...,...
87,22,5,238.0134,5274.0213
88,23,2,183.8715,1661.2590
89,23,3,182.1562,2830.4074
90,23,4,189.5532,3955.9252


In [51]:
df2.to_csv("breaks_data.csv", index=False)

## Final questionnaire answers

In [52]:
file_url = (
    'https://cloud.ilabt.imec.be/remote.php/dav/files/'+
    'af741990-37f9-103d-9441-9bec5c4808a7/ExperimentsData/'+
    'risky_dm_'+str(exp_version)+'/ExperimentData/final_questionnaire_answers.csv'
)


# Make a request to get the file content
response = requests.get(file_url, auth=(username, password))

# Check if the request was successful
if response.status_code == 200:
    # Stream the content directly into a pandas DataFrame
    csv_content = StringIO(response.text)
    df_final_q = pd.read_csv(csv_content)

    # Now `df` is your DataFrame containing the CSV data
    print("Final Questionnaire downloaded")  # Example: print the first few rows
else:
    print(f"Failed to retrieve the file. Status code: {response.status_code}")

df_final_q = df_final_q[df_final_q['ParticipantID'].isin(par_ids)]

Final Questionnaire downloaded


In [53]:
df_final_q["ParticipantID"] = df_final_q["ParticipantID"].replace(id_map)

# 1) Drop the first three columns
cols_to_drop = ["SessionID", "StudyID"]
df2 = df_final_q.drop(columns=cols_to_drop, errors="ignore")


/tmp/ipykernel_16993/3142224589.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final_q["ParticipantID"] = df_final_q["ParticipantID"].replace(id_map)


In [54]:
df2.to_csv("final_questionnaire_answers.csv", index=False)

## Full screen messages

In [55]:
# Define the path to your CSV file
if exp_version == 2:
    file_url = (
        'https://cloud.ilabt.imec.be/remote.php/dav/files/'+
        'af741990-37f9-103d-9441-9bec5c4808a7/ExperimentsData/'+
        'risky_dm_'+str(exp_version)+'/ExperimentData/full_screen_messages.csv'
    )
    
else:
    file_url = (
        'https://cloud.ilabt.imec.be/remote.php/dav/files/'+
        'af741990-37f9-103d-9441-9bec5c4808a7/ExperimentsData/'+
        'risky_dm_'+str(exp_version)+'/pilot/full_screen_messages.csv'
    )

# Make a request to get the file content
response = requests.get(file_url, auth=(username, password))

# Check if the request was successful
if response.status_code == 200:
    # Stream the content directly into a pandas DataFrame
    csv_content = StringIO(response.text)
    df_full_screen = pd.read_csv(csv_content)

    # Now `df` is your DataFrame containing the CSV data
    print("Full screen message downloaded")  # Example: print the first few rows
else:
    print(f"Failed to retrieve the file. Status code: {response.status_code}")

# Display the first few rows of the DataFrame
print(df_full_screen.columns)

Full screen message downloaded
Index(['StudyID', 'SessionID', 'ParticipantID', 'ParticipantID_timestamp',
       'TimeFullScreenMessageAppeared', 'ExperimentRoundNumber',
       'WhileInRound'],
      dtype='object')


In [56]:
df_full_screen = df_full_screen[df_full_screen['ParticipantID'].isin(par_ids)]

In [57]:
df_full_screen

,StudyID,SessionID,ParticipantID,ParticipantID_timestamp,TimeFullScreenMessageAppeared,ExperimentRoundNumber,WhileInRound
0,670cd6fec4b9b923bf4d92dd,670cd9c5c773e51a8840bad7,590df20e12802600017e1006,590df20e12802600017e1006_6569.7695,6569.7695,162,0
1,66d9bf36bfbd262851fca662,67039f799780ebcf741bc3df,5ae33e48ed7f8b0001ca7bf8,5ae33e48ed7f8b0001ca7bf8_90.49339999997616,90.4934,2,0
2,670cd6fec4b9b923bf4d92dd,670cdaa4f6695f5b545f3b6d,5b1d557444127b00013fec9e,5b1d557444127b00013fec9e_1234.28,1234.2800,34,0
3,670cd6fec4b9b923bf4d92dd,670cdaa4f6695f5b545f3b6d,5b1d557444127b00013fec9e,5b1d557444127b00013fec9e_27.682799999952316,27.6828,2,0
4,670cd6fec4b9b923bf4d92dd,670cdaa4f6695f5b545f3b6d,5b1d557444127b00013fec9e,5b1d557444127b00013fec9e_39.82970000004769,39.8297,2,0
5,670cd6fec4b9b923bf4d92dd,670cdaa4f6695f5b545f3b6d,5b1d557444127b00013fec9e,5b1d557444127b00013fec9e_5379.970900000096,5379.9709,162,0
6,670cd6fec4b9b923bf4d92dd,670cdaa4f6695f5b545f3b6d,5b1d557444127b00013fec9e,5b1d557444127b00013fec9e_63.15929999995232,63.1593,2,0
7,670cd6fec4b9b923bf4d92dd,670cd9612caf5d5888709f6d,5bcedec641f0710001c74dd5,5bcedec641f0710001c74dd5_29.745600000023842,29.7456,2,0
8,670cd6fec4b9b923bf4d92dd,670cda96564dd56c1acf3589,5bd0a57f446a8e0001204390,5bd0a57f446a8e0001204390_20.487699999999254,20.4877,2,0
9,66d9bf36bfbd262851fca662,67039d0053f5a09f3c8e6e0c,5be2fc5f0d71c80001ff4cb2,5be2fc5f0d71c80001ff4cb2_261.61789999997615,261.6179,2,0


In [64]:
# Replace participant IDs

df = df_full_screen
df["ParticipantID"] = df["ParticipantID"].replace(id_map)

# 1) Drop the first three columns
cols_to_drop = ["SessionID", "StudyID", 'ParticipantID_timestamp']
df2 = df.drop(columns=cols_to_drop, errors="ignore")

# 2) Ensure numeric sort keys, then sort by participant → round → (optionally) gamble
df2['ExperimentRoundNumber'] = pd.to_numeric(df2['ExperimentRoundNumber'], errors="coerce")
df2['TimeFullScreenMessageAppeared'] = pd.to_numeric(df2['TimeFullScreenMessageAppeared'], errors="coerce")
# If ParticipantID is numeric-like (e.g., 1..23); if it's strings like "P001", remove this next line
df2["ParticipantID"] = pd.to_numeric(df2["ParticipantID"], errors="ignore")

df2 = df2.sort_values(
    by=["ParticipantID", 'ExperimentRoundNumber','TimeFullScreenMessageAppeared'],
    kind="mergesort"  # stable sort keeps within-round row order if ties
).reset_index(drop=True)

# 3) Reorder columns: put key columns first, keep the rest as-is
first = ["ParticipantID", "ExperimentRoundNumber", "TimeFullScreenMessageAppeared"]
df2 = df2[first + [c for c in df2.columns if c not in first]]

/tmp/ipykernel_16993/309588782.py:14: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df2["ParticipantID"] = pd.to_numeric(df2["ParticipantID"], errors="ignore")


In [65]:
df2

,ParticipantID,ExperimentRoundNumber,TimeFullScreenMessageAppeared,WhileInRound
0,1,162,6569.7695,0
1,2,2,90.4934,0
2,3,2,27.6828,0
3,3,2,39.8297,0
4,3,2,63.1593,0
5,3,34,1234.2800,0
6,3,162,5379.9709,0
7,5,2,29.7456,0
8,6,2,20.4877,0
9,7,2,261.6179,0


In [66]:
df2.to_csv("full_screen_messages.csv", index=False)


## Instructions_completion_times

In [67]:
# Define the path to your CSV file
if exp_version == 2:
    file_url = (
        'https://cloud.ilabt.imec.be/remote.php/dav/files/'+
        'af741990-37f9-103d-9441-9bec5c4808a7/ExperimentsData/'+
        'risky_dm_'+str(exp_version)+'/ExperimentData/instructions_completion_times.csv'
    )
    
else:
    file_url = (
        'https://cloud.ilabt.imec.be/remote.php/dav/files/'+
        'af741990-37f9-103d-9441-9bec5c4808a7/ExperimentsData/'+
        'risky_dm_'+str(exp_version)+'/pilot/instructions_completion_times.csv'
    )

# Make a request to get the file content
response = requests.get(file_url, auth=(username, password))

# Check if the request was successful
if response.status_code == 200:
    # Stream the content directly into a pandas DataFrame
    csv_content = StringIO(response.text)
    df_instructions = pd.read_csv(csv_content)

    # Now `df` is your DataFrame containing the CSV data
    print("Instructions downloaded")  # Example: print the first few rows
else:
    print(f"Failed to retrieve the file. Status code: {response.status_code}")

# Display the first few rows of the DataFrame
print(df_instructions.columns)

Instructions downloaded
Index(['StudyID', 'SessionID', 'ParticipantID', 'CompletionTimeConsentForm',
       'CompletionTimeIntroduction', 'CompletionTimeInstruction1',
       'CompletionTimeInstruction2', 'CompletionTimeInstruction3',
       'CompletionTimeInstruction4', 'CompletionTimeInstruction5',
       'CompletionTimeInstruction6', 'CompletionTimeInstructionsCompleted',
       'CompletionTimeTimingInstructions', 'CompletionTimeAllInstructions',
       'CompletionTimeComprehensionCheck1',
       'CompletionTimeComprehensionCheck2',
       'CompletionTimeComprehensionCheck3',
       'CompletionTimeComprehensionCheck4', 'AttemptsLeftComprehensionCheck1',
       'AttemptsLeftComprehensionCheck2', 'AttemptsLeftComprehensionCheck3',
       'AttemptsLeftComprehensionCheck4', 'VisitsConsentForm',
       'VisitsInstruction1', 'VisitsInstruction2', 'VisitsInstruction3',
       'VisitsInstruction4', 'VisitsInstruction5', 'VisitsInstruction6'],
      dtype='object')


In [68]:
df_instructions = df_instructions[df_instructions['ParticipantID'].isin(par_ids)]

In [69]:

df_instructions["ParticipantID"] = df_instructions["ParticipantID"].replace(id_map)

# 1) Drop the first three columns
cols_to_drop = ["SessionID", "StudyID"]
df2 = df_instructions.drop(columns=cols_to_drop, errors="ignore")

/tmp/ipykernel_16993/2329584756.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_instructions["ParticipantID"] = df_instructions["ParticipantID"].replace(id_map)


In [70]:
df2

,ParticipantID,CompletionTimeConsentForm,CompletionTimeIntroduction,CompletionTimeInstruction1,CompletionTimeInstruction2,CompletionTimeInstruction3,CompletionTimeInstruction4,CompletionTimeInstruction5,CompletionTimeInstruction6,CompletionTimeInstructionsCompleted,...,AttemptsLeftComprehensionCheck2,AttemptsLeftComprehensionCheck3,AttemptsLeftComprehensionCheck4,VisitsConsentForm,VisitsInstruction1,VisitsInstruction2,VisitsInstruction3,VisitsInstruction4,VisitsInstruction5,VisitsInstruction6
0,1,223.0024,47.8742,308.6497,50.7557,38.3700,107.6580,28.8604,25.9794,25.7130,...,3,3,2,1,2,3,1,1,1,1
1,2,90.4654,29.6569,52.7287,18.0866,17.0695,65.4665,14.5864,4.8674,7.5672,...,2,3,3,1,1,1,1,1,1,1
2,3,27.6506,26.9272,83.1153,1.7991,8.2069,28.9503,12.9183,5.7117,1.5093,...,3,3,3,1,1,1,3,1,1,1
3,4,10.0412,2.9157,8.7710,2.4242,24.2602,131.9913,39.8449,23.2378,9.9565,...,3,3,1,1,1,1,1,1,1,1
4,5,29.6851,16.5348,43.4136,30.4632,29.3831,49.5427,16.4710,3.5993,8.7270,...,3,3,3,1,1,1,1,1,1,1
5,6,20.0354,25.1148,72.7909,27.8389,23.6073,62.4192,26.1020,10.1506,13.2929,...,3,3,3,1,1,1,1,1,1,1
6,7,261.5803,38.6569,155.9930,78.3077,60.2654,171.8722,48.7837,24.6696,28.5536,...,3,3,3,1,1,1,1,1,1,1
7,8,170.2841,18.4470,63.3079,56.1256,60.4677,145.7170,44.7490,31.3172,12.5179,...,3,3,3,1,1,1,2,1,1,1
8,9,34.9823,24.3021,64.1612,38.8941,38.9316,74.8282,20.0846,10.6865,12.0887,...,1,3,2,1,1,1,1,1,1,1
9,10,357.5457,34.8596,99.9267,61.8123,121.3549,156.6735,61.8878,32.7639,25.8973,...,3,3,3,1,1,1,1,1,1,1


In [71]:
df2.to_csv("instructions_completion_times.csv", index=False)

## Mouse tracking data

In [74]:
# Define the path to your CSV file
if exp_version == 2:
    file_url = (
        'https://cloud.ilabt.imec.be/remote.php/dav/files/'+
        'af741990-37f9-103d-9441-9bec5c4808a7/ExperimentsData/'+
        'risky_dm_'+str(exp_version)+'/ExperimentData/mouse_coordinates.csv'
    )
    
else:
    file_url = (
        'https://cloud.ilabt.imec.be/remote.php/dav/files/'+
        'af741990-37f9-103d-9441-9bec5c4808a7/ExperimentsData/'+
        'risky_dm_'+str(exp_version)+'/pilot/mouse_coordinates.csv'
    )

# Make a request to get the file content
response = requests.get(file_url, auth=(username, password))

# Check if the request was successful
if response.status_code == 200:
    # Stream the content directly into a pandas DataFrame
    csv_content = StringIO(response.text)
    df_mouse_coordinates = pd.read_csv(csv_content)

    # Now `df` is your DataFrame containing the CSV data
    print("Mouse coordinates downloaded")  # Example: print the first few rows
else:
    print(f"Failed to retrieve the file. Status code: {response.status_code}")

# Display the first few rows of the DataFrame
print(df_mouse_coordinates.columns)

Mouse coordinates downloaded
Index(['StudyID', 'SessionID', 'ExperimentRoundID', 'ParticipantID',
       'ExperimentGambleNumber', 'ExperimentRoundNumber', 'MouseXCoordinates',
       'MouseYCoordinates', 'RectLeftOptionLeftXCoord',
       'RectLeftOptionRightXCoord', 'RectLeftOptionTopYCoord',
       'RectLeftOptionBottomYCoord', 'RectRightOptionLeftXCoord',
       'RectRightOptionRightXCoord', 'RectRightOptionTopYCoord',
       'RectRightOptionBottomYCoord', 'RectStartButtonLeftXCoord',
       'RectStartButtonRightXCoord', 'RectStartButtonTopYCoord',
       'RectStartButtonBottomYCoord', 'BigRectTopYCoord',
       'BigRectBottomYCoord', 'BigRectLeftXCoord', 'BigRectRightXCoord'],
      dtype='object')


In [75]:
df_mouse_coordinates.head

<bound method NDFrame.head of                        StudyID                 SessionID  \
0     670cd6fec4b9b923bf4d92dd  670cd9c5c773e51a8840bad7   
1     670cd6fec4b9b923bf4d92dd  670cd9c5c773e51a8840bad7   
2     670cd6fec4b9b923bf4d92dd  670cd9c5c773e51a8840bad7   
3     670cd6fec4b9b923bf4d92dd  670cd9c5c773e51a8840bad7   
4     670cd6fec4b9b923bf4d92dd  670cd9c5c773e51a8840bad7   
...                        ...                       ...   
4086  670cd6fec4b9b923bf4d92dd  670cd913e25b59a2a5f2e385   
4087  670cd6fec4b9b923bf4d92dd  670cd913e25b59a2a5f2e385   
4088  670cd6fec4b9b923bf4d92dd  670cd913e25b59a2a5f2e385   
4089  670cd6fec4b9b923bf4d92dd  670cd913e25b59a2a5f2e385   
4090  670cd6fec4b9b923bf4d92dd  670cd913e25b59a2a5f2e385   

                            ExperimentRoundID             ParticipantID  \
0       590df20e12802600017e1006_1_experiment  590df20e12802600017e1006   
1      590df20e12802600017e1006_10_experiment  590df20e12802600017e1006   
2     590df20e1280260001

In [76]:
df_mouse_coordinates = df_mouse_coordinates[df_mouse_coordinates['ParticipantID'].isin(par_ids)]
df_mouse_coordinates["ParticipantID"] = df_mouse_coordinates["ParticipantID"].replace(id_map)

/tmp/ipykernel_16993/589082987.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_mouse_coordinates["ParticipantID"] = df_mouse_coordinates["ParticipantID"].replace(id_map)
/tmp/ipykernel_16993/589082987.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mouse_coordinates["ParticipantID"] = df_mouse_coordinates["ParticipantID"].replace(id_map)


In [78]:

cols_to_drop = ["SessionID", "StudyID", "ExperimentRoundID"]
df2 = df_mouse_coordinates.drop(columns=cols_to_drop, errors="ignore")

# 2) Ensure numeric sort keys, then sort by participant → round → (optionally) gamble
df2["ExperimentRoundNumber"] = pd.to_numeric(df2["ExperimentRoundNumber"], errors="coerce")
# If ParticipantID is numeric-like (e.g., 1..23); if it's strings like "P001", remove this next line
df2["ParticipantID"] = pd.to_numeric(df2["ParticipantID"], errors="ignore")

df2 = df2.sort_values(
    by=["ParticipantID", "ExperimentRoundNumber"],
    kind="mergesort"  # stable sort keeps within-round row order if ties
).reset_index(drop=True)

# 3) Reorder columns: put key columns first, keep the rest as-is
first = ["ParticipantID", "ExperimentRoundNumber", "ExperimentGambleNumber"]
df2 = df2[first + [c for c in df2.columns if c not in first]]

/tmp/ipykernel_16993/365971526.py:7: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df2["ParticipantID"] = pd.to_numeric(df2["ParticipantID"], errors="ignore")


In [79]:
df2.columns

Index(['ParticipantID', 'ExperimentRoundNumber', 'ExperimentGambleNumber',
       'MouseXCoordinates', 'MouseYCoordinates', 'RectLeftOptionLeftXCoord',
       'RectLeftOptionRightXCoord', 'RectLeftOptionTopYCoord',
       'RectLeftOptionBottomYCoord', 'RectRightOptionLeftXCoord',
       'RectRightOptionRightXCoord', 'RectRightOptionTopYCoord',
       'RectRightOptionBottomYCoord', 'RectStartButtonLeftXCoord',
       'RectStartButtonRightXCoord', 'RectStartButtonTopYCoord',
       'RectStartButtonBottomYCoord', 'BigRectTopYCoord',
       'BigRectBottomYCoord', 'BigRectLeftXCoord', 'BigRectRightXCoord'],
      dtype='object')

In [80]:
df2.to_csv("mouse_coordinates.csv", index=False)

In [81]:
df2.head

<bound method NDFrame.head of       ParticipantID  ExperimentRoundNumber  ExperimentGambleNumber  \
0                 1                      1                     135   
1                 1                      2                      19   
2                 1                      3                     112   
3                 1                      4                      89   
4                 1                      5                      99   
...             ...                    ...                     ...   
3670             23                    156                      70   
3671             23                    157                     141   
3672             23                    158                     102   
3673             23                    159                      26   
3674             23                    160                     143   

                                      MouseXCoordinates  \
0                                                   NaN   
1          